In [1]:
import os
import sys
srcpath = os.path.realpath('SourceFiles')
sys.path.append(srcpath)

In [2]:
from teproteus import TEProteusAdmin as TepAdmin
admin = TepAdmin() #required to control PXI module
sid = 8 #PXI slot WDS found
inst = admin.open_instrument(slot_id=sid)
resp = inst.send_scpi_query("*IDN?")
print('connected to: ' + resp) # Print *IDN

Load TEProteus.dll from : C:\Windows\system32\TEProteus.dll
connected to: Tabor Electronics,P9484M,000002243278,1.237.0 --slot#: 8,


In [3]:
import matplotlib.pyplot as plt
import numpy as np
sampleRateDAC = 1E9
amp = 1
cycles = 10
segLen = 1024 # must be a multiple of 64
time = np.linspace(0, segLen-1, segLen)
w = 2 * np.pi * cycles
dacWave = amp * np.sin(w*time/segLen)
print('Frequency {0} Hz'.format(sampleRateDAC*cycles/segLen))

Frequency 9765625.0 Hz


In [4]:
max_dac=65535 # Max Dac 
half_dac=max_dac/2 # DC Level 
data_type = np.uint16 # DAC data type  
dacWave = ((dacWave) + 1.0) * half_dac   
dacWave = dacWave.astype(data_type)  

In [5]:
inst.send_scpi_cmd('*CLS; *RST') # Init instrument 
#AWG channel 
ch = 1 # everything after relates to CH 1 
cmd = ':INST:CHAN {0}'.format(ch) 
inst.send_scpi_cmd(cmd) 
cmd = ':FREQ:RAST {0}'.format(sampleRateDAC)  
inst.send_scpi_cmd(cmd) 
cmd = ':TRAC:DEL:ALL' # Clear CH 1 Memory 
inst.send_scpi_cmd(cmd) 
cmd = ':INIT:CONT ON' # play waveform continuously 
inst.send_scpi_cmd(cmd)

0

In [6]:
segnum = 1 
cmd = ':TRAC:DEF {0}, {1}'.format(segnum, len(dacWave)) # memory location and length 
inst.send_scpi_cmd(cmd) 
# Select the segment 
cmd = ':TRAC:SEL {0}'.format(segnum) 
inst.send_scpi_cmd(cmd)

0

In [8]:
inst.timeout = 30000 #increase 
inst.write_binary_data('*OPC?; :TRAC:DATA', dacWave) # write, and wait while *OPC completes 
inst.timeout = 10000 # return to normal 
cmd = ':FUNC:MODE:SEGM {0}'.format(segnum) 
inst.send_scpi_cmd(cmd) 
cmd = ':OUTP ON' 
rc = inst.send_scpi_cmd(cmd)

In [9]:
inst.send_scpi_cmd(cmd)
cmd = ':TRAC:DEL:ALL' # Clear CH 1 Memory
inst.send_scpi_cmd(cmd)
cmd = ':INIT:CONT ON' # play waveform continuously

In [10]:
inst.send_scpi_cmd('*CLS; *RST') # Init instrument

0

In [1]:
array1 = [1, 3, 5]
array2 = [2, 4, 6]

# 使用zip交替合并两个数组
merged_array = [val for pair in zip(array1, array2) for val in pair]

print(merged_array)

[1, 2, 3, 4, 5, 6]


In [2]:
0.5*64

32.0

In [9]:
ncoFreq= 25E6  #<---- CF
sampleRateDAC = 1.125E9
ncoFreq= 3E6

 

#wavefore parameters

max_dac=65535 # Max Dac

half_dac=max_dac/2 # DC Level

data_type = np.uint16 # DAC data type

 

onLen = 5 # Pulse on time multiple of 32 or 28ns

 

segTime = onLen / (sampleRateDAC)

print('On time')

print(segTime)

 

 

dacWaveOn_I = np.ones(onLen)

dacWaveOn_I = dacWaveOn_I * max_dac

dacWaveOn_I = dacWaveOn_I.astype(data_type)

dacWaveOn_Q = dacWaveOn_I*0

# reshapes for IQIQIQIQIQ....

arr_tuple = (dacWaveOn_I, dacWaveOn_Q)

dacWaveOn_IQ = np.vstack(arr_tuple).reshape((-1,), order='F')
print(dacWaveOn_IQ)

On time
4.444444444444444e-09
[65535     0 65535     0 65535     0 65535     0 65535     0]
